# Unit 5 - Financial Planning

In [7]:
# Initial imports
import os
import requests
import pandas as pd
#loading .env enviorment variables
from dotenv import load_dotenv
#pulling stock returns from alpaca that will be used to determine historical trends 
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
#JSON is used to convert the dictionary into a JSON string, which can then be written into a file
import json
#enable the inline plotting
%matplotlib inline

ModuleNotFoundError: No module named 'MCForecastTools'

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
#Use the requests library to fetch the current price in Canadian dollars (CAD) of bitcoin (BTC) and ethereum (ETH) 
#Fetch current BTC price and parce json file
btc_API = requests.get(btc_url).json()
btc_value = btc_API["data"]["1"]["quotes"]["CAD"]["price"]

# Fetch current ETH price and parce json file 
eth_API = requests.get(eth_url).json()
#reading the correct value in the json file inside data dictionary 
eth_value = eth_API["data"]["1027"]["quotes"]["CAD"]["price"]

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
#defining and assigning bond amount 
my_agg = 200
#defining and assigning the stock amount 
my_spy = 50

In [ ]:
#create a .env file in your working directory to store the values of your Alpaca API key and Alpaca secret key.
#In this case the API Key and the Secret Key are the same thing and are both stored in the .env file 
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

# create the Alpaca API object using the tradeapi.REST function from the Alpaca SDK.
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    #this just referrs to the version of alpaca being used 
    api_version="v2"
)

In [ ]:
# Format current date as ISO format
# ISO format follows the YYYY-MM-DDTHH:MM:SS. mmmmmm format
current_date = pd.Timestamp("2021-09-16", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["AGG", "SPY"]
# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"
# Get current closing prices for SPY and AGG from the currentdate timestamp 
df_current_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = current_date,
    #1000 trading days going back in history 
    limit = 1000
).df
# Reorganize the DataFrame
# Separate ticker data
AGG = df_current_portfolio[df_current_portfolio['symbol']=='AGG'].drop('symbol', axis=1)
SPY = df_current_portfolio[df_current_portfolio['symbol']=='SPY'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames, this means to link or connect the dataframes together and then print
df_current_portfolio = pd.concat([AGG, SPY],axis=1, keys=['AGG','SPY'])

# Preview DataFrame
df_current_portfolio

In [ ]:
#calling in the first row as that contains the closing price for both AGG and SPY and displaying that
agg_price = float(df_current_portfolio["AGG"]["close"][0])
spy_price = float(df_current_portfolio["SPY"]["close"][0])
print(f"Current AGG closing price: ${agg_price}")
print(f"Current SPY closing price: ${spy_price}")

In [ ]:
# Compute the current value of shares
agg_portfolio = my_agg * agg_price
spy_portfolio = my_spy * spy_price
# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${agg_portfolio:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${spy_portfolio:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
data = {
    "Amount": [19385.986877, 40616.5000000]
}

# Create savings DataFrame
tickers = ["Crypto", "Shares"]
df_savings = pd.DataFrame(data, index = tickers)

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart to visualize the composition of personal savings
df_savings.plot.pie(y="Amount", title="Personal Savings Composition")

In [8]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_amount_savings = 19385.986877 + 40616.5000000

# Validate saving health by creating an loop that will output a different message depending on the calculation
if total_amount_savings > emergency_fund:
    print('Congrats! You have enough money in this fund')

elif total_amount_savings == emergency_fund:
    print ('Congrats on reaching your financial goal')

elif total_amount_savings < emergency_fund:
    dollar_need_to_save = emergency_fund - total_amount_savings
    print ('To reach your goal you need to save: $')
    print (dollar_need_to_save)

Congrats! You have enough money in this fund


## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Format current date as ISO format
# ISO format follows the YYYY-MM-DDTHH:MM:SS. mmmmmm format
current_date = pd.Timestamp("2021-09-16", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]
# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

df_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = current_date,
    #will give you 5 years going back from the current date  
    limit = 1260
).df
# Reorganize the DataFrame
# Separate ticker data
AGG = df_stock_data[df_stock_data['symbol']=='AGG'].drop('symbol', axis=1)
SPY = df_stock_data[df_stock_data['symbol']=='SPY'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames, this means to link or connect the dataframes together and then print
df_stock_data = pd.concat([AGG, SPY],axis=1, keys=['AGG','SPY'])

# Preview DataFrame
df_stock_data

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# The homework outlined that the number of simulations is 500 and the split (weights) is 40/60
MC_dist = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
)

In [ ]:
# Printing the simulation input data
MC_dist.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_dist.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
plotted_outcomes = MC_dist.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
plotted_dist_plot = MC_dist.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_tbl = MC_dist.summarize_cumulative_return()

# Print summary statistics
print(summary_tbl)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# is the decminal roud off 
ci_lower = round(summary_tbl[8]*20000,2)
ci_upper = round(summary_tbl[9]*20000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000

ci_lower = round(summary_tbl[8]*initial_investment)
ci_upper = round(summary_tbl[9]*initial_investment)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns


In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!


In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")